In [10]:
!pip install binance

In [219]:
!pip install schedule

  Using cached schedule-1.2.2-py3-none-any.whl.metadata (3.8 kB)
Using cached schedule-1.2.2-py3-none-any.whl (12 kB)


In [1]:
import ccxt
import requests
from binance.client import Client
from binance.exceptions import BinanceAPIException
from datetime import datetime, timedelta
import time


In [2]:
def information_for_options(client):
    markets = client.fetch_markets()
    server_time = client.fetch_time()
    # Filter option markets
    option_markets = [market for market in markets if market['type'] == 'option']
    # print("Option markets:", option_markets)
    for i in option_markets:
        # print(i["info"])
        filtered_list = []
        expiry_timestamp = float(i['info']['expiration_timestamp'])/1000
        # Convert expiry timestamp to datetime
        expiry_datetime = datetime.utcfromtimestamp(expiry_timestamp)

        # Calculate current datetime plus 20 weeks
        current_datetime = datetime.utcnow()
        twenty_weeks_later = current_datetime + timedelta(weeks=20)
        # Check if expiry datetime is greater than 20 weeks later
        if expiry_datetime <= twenty_weeks_later:
            filtered_list.append(i)
            # if call option and strike is not none, add to list
            second_filtered_list = []
            for item in filtered_list:
                # if item["strike"] is not None and 
                if item["optionType"] == "call" and item["strike"] is not None:

                    second_filtered_list.append(item)
                    # third_filtered_list = []
                    # # Loop through second filtered list and get contract with longest expiration
                    # for item in second_filtered_list:
                    #     expired_time = item["expiration_timestamp"]
                        
    #                     if item["expiration_timestamp"] is not None:
    #                         third_filtered_list.append(item)
            third_filtered_list = []
            expiry_timestamp_list = []
            if len(second_filtered_list) > 0:
                longest_expiry_contract = max(second_filtered_list, key=lambda x: x['expiry'])

    # print(longest_expiry_contract)
    # Contract symbol
    symbol = longest_expiry_contract["id"]
    # Timestamp of contract            
    expire_time = float(longest_expiry_contract["info"]["expiration_timestamp"])
    # convert expiration timestamp to datetime
    expiry_datetime = datetime.utcfromtimestamp(expire_time/1000)
    # Get delta
    contract_delta = client.fetch_greeks(longest_expiry_contract["symbol"])
    delta = contract_delta["info"]["greeks"]["delta"]

    # Get contract size
    # Calculate share to purchase
    contract_size = longest_expiry_contract["info"]["contract_size"]
    share_to_purchase = float(delta) * float(contract_size)


    return expiry_datetime, share_to_purchase, symbol ,delta ,contract_size

In [3]:

mode = "test"

client_id_testnet = "m_cYdGRs"
client_secret_testnet = "CFMj4XERdBpYU5f-xGiEeM6q29WWPh78KbOllO8I5nI"

client_id_realnet = "VUSrWKNX"
client_secret_realnet = "CNIEmjiKy2p-h28O4Mda1QKD8hXJ3duA5rAODdLfvwE"

if mode == "test":
    client_id = client_id_testnet
    client_secret = client_secret_testnet
    base_url = 'https://test.deribit.com/api/v2/' # test base url
else:
    client_id = client_id_realnet
    client_secret = client_secret_realnet
    base_url = 'https://www.deribit.com/api/v2/'

print(base_url)


# Function to get access token
def get_access_token(client_id, client_secret):
    url = f'{base_url}public/auth'
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.get(url, params=params)
    response_data = response.json()
    if 'result' in response_data:
        return response_data['result']['access_token']
    else:
        raise Exception('Failed to get access token: ' + response_data.get('error', {}).get('message', 'Unknown error'))

# Function to fetch account information
def fetch_account_info(access_token):
    url = f'{base_url}private/get_account_summary'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'currency': 'BTC'  # or 'ETH', etc.
    }
    response = requests.get(url, headers=headers, params=params)
    response_data = response.json()
    if 'result' in response_data:
        return response_data['result']
    else:
        raise Exception('Failed to fetch account information: ' + response_data.get('error', {}).get('message', 'Unknown error'))
    
client = ccxt.deribit({
    'apiKey': client_id,
    'secret': client_secret,
    'timeout': 50000,
})
if mode=="test":
    client.set_sandbox_mode(True)

https://test.deribit.com/api/v2/


In [4]:
access_token = get_access_token(client_id, client_secret)
account_info = fetch_account_info(access_token)
print(account_info)

{'initial_margin': 0.0, 'options_vega_map': {}, 'portfolio_margining_enabled': False, 'estimated_liquidation_ratio_map': {}, 'currency': 'BTC', 'delta_total': 0.0, 'futures_pl': 0.0, 'spot_reserve': 0.0, 'options_session_upl': 0.0, 'additional_reserve': 0.0, 'available_withdrawal_funds': 100.0, 'equity': 100.0, 'margin_balance': 100.0, 'projected_initial_margin': 0.0, 'options_gamma': 0.0, 'session_upl': 0.0, 'limits': {'non_matching_engine': {'burst': 15000, 'rate': 20}, 'limits_per_currency': False, 'matching_engine': {'trading': {'total': {'burst': 20, 'rate': 5}}, 'spot': {'burst': 20, 'rate': 5}, 'maximum_quotes': {'burst': 500, 'rate': 500}, 'maximum_mass_quotes': {'burst': 10, 'rate': 10}, 'guaranteed_mass_quotes': {'burst': 2, 'rate': 2}, 'cancel_all': {'burst': 20, 'rate': 5}}}, 'options_theta_map': {}, 'futures_session_rpl': 0.0, 'futures_session_upl': 0.0, 'options_theta': 0.0, 'estimated_liquidation_ratio': 0.0, 'options_vega': 0.0, 'options_delta': 0.0, 'delta_total_map': 

In [5]:

def create_option_order(client):
    try:
        # Construct the order
        symbol=information_for_options(client)[-3]
        order = client.create_market_buy_order(symbol,1)

        print(f"Order created successfully: {order}")
        return order

    except Exception as e:
        print(f"Failed to create order: {e}")
        return None

# Example usage

In [6]:
expiry_datetime, share_to_purchase, symbol ,delta ,contract_size= information_for_options(client)
print(expiry_datetime,share_to_purchase, symbol)
create_option_order(client)

2024-09-27 08:00:00 0.01289 ETH-27SEP24-17000-C
Order created successfully: {'info': {'is_liquidation': False, 'risk_reducing': False, 'creation_timestamp': '1718212376750', 'order_type': 'limit', 'order_state': 'open', 'contracts': '1.0', 'original_order_type': 'market', 'average_price': '0.0', 'reduce_only': False, 'last_update_timestamp': '1718212376750', 'filled_amount': '0.0', 'post_only': False, 'replaced': False, 'web': False, 'api': True, 'mmp': False, 'instrument_name': 'ETH-27SEP24-17000-C', 'order_id': 'ETH-8807922265', 'max_show': '1.0', 'time_in_force': 'good_til_cancelled', 'direction': 'buy', 'amount': '1.0', 'price': '0.036', 'label': '', 'trades': []}, 'id': 'ETH-8807922265', 'clientOrderId': None, 'timestamp': 1718212376750, 'datetime': '2024-06-12T17:12:56.750Z', 'lastTradeTimestamp': None, 'symbol': 'ETH/USD:ETH-240927-17000-C', 'type': 'limit', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'buy', 'price': 0.036, 'stopPrice': None, 'triggerPrice': None, 'amount':

{'info': {'is_liquidation': False,
  'risk_reducing': False,
  'creation_timestamp': '1718212376750',
  'order_type': 'limit',
  'order_state': 'open',
  'contracts': '1.0',
  'original_order_type': 'market',
  'average_price': '0.0',
  'reduce_only': False,
  'last_update_timestamp': '1718212376750',
  'filled_amount': '0.0',
  'post_only': False,
  'replaced': False,
  'web': False,
  'api': True,
  'mmp': False,
  'instrument_name': 'ETH-27SEP24-17000-C',
  'order_id': 'ETH-8807922265',
  'max_show': '1.0',
  'time_in_force': 'good_til_cancelled',
  'direction': 'buy',
  'amount': '1.0',
  'price': '0.036',
  'label': '',
  'trades': []},
 'id': 'ETH-8807922265',
 'clientOrderId': None,
 'timestamp': 1718212376750,
 'datetime': '2024-06-12T17:12:56.750Z',
 'lastTradeTimestamp': None,
 'symbol': 'ETH/USD:ETH-240927-17000-C',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 0.036,
 'stopPrice': None,
 'triggerPrice': None,
 'amount': 1.0,
 'cost':

In [7]:
share_to_purchase,symbol,delta,contract_size

(0.01289, 'ETH-27SEP24-17000-C', '0.01289', '1.0')

In [8]:
symbol=symbol.split("-")[0]
symbol=symbol+"USDT"
symbol

'ETHUSDT'

In [34]:
def create_hedging_order(symbol, side,share_to_purchase):
    """
    Create a hedging order on Binance testnet.

    Parameters:
    api_key (str): Your Binance testnet API key.
    api_secret (str): Your Binance testnet API secret.
    symbol (str): The trading pair symbol (e.g., 'BTCUSDT').
    side (str): 'BUY' or 'SELL' order.
    share_to_purchase (float): Amount of the asset to purchase.

    Returns:
    dict: Response from Binance API with order details.
    """
    # Connect to Binance testnet
    api_key = 'ZQatsB1ChLN5LDHH2fMr7rj9lgiYBLs5NlF1HgagdyepcoOAYnV3kDsVXGkXKrgb'
    api_secret = 'aM9Jzg7gp6LKiKxzXB7H7F0IPj72D33KiDZr8Own4ByvRRTqUTzVeXbfmDIcFx9b'
    client = Client(api_key, api_secret, testnet=True)
    # Get current price of the symbol
    ticker = client.get_symbol_ticker(symbol=symbol)
    current_price = float(ticker['price'])
    print(f"Current price of {symbol}: {current_price}")
    print(f"Share to purchase: {share_to_purchase} ")
    # Create a market order
    try:
        order = client.create_order(
            symbol=symbol,
            side=side,
            type='MARKET',
            quantity=float (share_to_purchase)
        )
        print("Order placed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        order = None
    
    return order


# Example usage:

side = 'BUY'  # or 'SELL'


In [27]:
order = create_hedging_order(symbol, side, share_to_purchase)
print(order)

Current price of ETHUSDT: 3612.08
An error occurred: APIError(code=-1013): Filter failure: LOT_SIZE
None


In [28]:
information_for_options(client)

(datetime.datetime(2024, 9, 27, 8, 0),
 0.0009,
 'ETH-27SEP24-17000-C',
 '0.0009',
 '1.0')

In [29]:
type(information_for_options(client)[-2])

str

In [32]:
symbol

'ETHUSDT'

In [30]:
def calculate_and_place_order(symbol,old_delta):
    old_delta=float(old_delta)
    new_delta=float(information_for_options(client)[-2])
    contract_size=float(information_for_options(client)[-1])
    share_to_purchase = (new_delta - old_delta) * contract_size
    if share_to_purchase != 0:  # Only place an order if there's a change in delta
        if new_delta > old_delta:
            side = "BUY"
        else:
            side = "SELL"
        create_hedging_order(symbol,side, abs(share_to_purchase))
    else:
        print(f"{datetime.now()}: No change in delta. No order placed.")


In [35]:
import schedule
import time

# Assuming these are updated dynamically from a data source

old_delta=0

def job():
    global old_delta
    print("Performing job")
    calculate_and_place_order(symbol,old_delta)
    old_delta=information_for_options(client)[-2]
    print(f"Updated old_delta to: {old_delta}")
# Schedule the job every hour
schedule.every(20).seconds.do(job)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)

Performing job
0
Current price of ETHUSDT: 3614.58
Share to purchase: 0.00089 
An error occurred: APIError(code=-1013): Filter failure: LOT_SIZE
Updated old_delta to: 0.0009
Performing job
2024-06-13 00:27:38.006587: No change in delta. No order placed.
Updated old_delta to: 0.0009
Performing job
0.0009
2024-06-13 00:27:42.504336: No change in delta. No order placed.
Updated old_delta to: 0.0009
Performing job
0.0009
Updated old_delta to: 0.0009
Performing job
2024-06-13 00:27:49.321612: No change in delta. No order placed.
Updated old_delta to: 0.0009
Performing job
0.0009
2024-06-13 00:27:54.970313: No change in delta. No order placed.
Updated old_delta to: 0.0009
Performing job
Current price of ETHUSDT: 3614.89
Share to purchase: 1.0000000000000026e-05 
An error occurred: APIError(code=-1100): Illegal characters found in parameter 'quantity'; legal range is '^([0-9]{1,20})(\.[0-9]{1,20})?$'.
Updated old_delta to: 0.0009
Performing job
0.0009
2024-06-13 00:28:08.079095: No change in 

KeyboardInterrupt: 